In [3]:
import glob
import pandas as pd
import numpy as np

import os
from os.path import join
import random

import lightgbm as lgb
from sklearn.model_selection import KFold

import time
import warnings

warnings.filterwarnings("ignore")

In [4]:
ROOT_DIR_PATH = os.getcwd()
TRAIN_FEATHER_PATH = 'C:\\Users\\User\\Documents\\not_for_git\\orbit_project\\fe_data\\train_mid.ftr'
TEST_FEATHER_PATH = 'C:\\Users\\User\\Documents\\not_for_git\\orbit_project\\fe_data\\test_mid.ftr'

def read_feather():
    train_df = pd.read_feather(TRAIN_FEATHER_PATH)
    test_df = pd.read_feather(TEST_FEATHER_PATH)
    return train_df, test_df

In [5]:
train_df, _ = read_feather()

In [6]:
train_df.head()

,temp1,temp2,temp3,temp4,temp5,temp6,temp7,temp8,temp9,long_GMI,lat_GMI,precipitation,orbit,subset,pixel,elevation,type_1,type_2,month,day
0,0.049494,0.034790,0.036629,0.050757,0.419546,0.153418,0.151771,0.867290,0.640426,159.494385,5.641016,0.0,10462,1,1,0.876454,0,0,1,1
1,0.049653,0.034238,0.037082,0.050690,0.416622,0.154150,0.148810,0.861778,0.631940,159.534912,5.609135,0.0,10462,1,2,0.876454,0,0,1,1
2,0.048762,0.035108,0.037101,0.050481,0.420030,0.153041,0.146607,0.860670,0.632753,159.575806,5.577742,0.0,10462,1,3,0.876454,0,0,1,1
3,0.049704,0.034355,0.034933,0.048831,0.409620,0.150611,0.141517,0.854340,0.622288,159.617081,5.546840,0.0,10462,1,4,0.876454,0,0,1,1
4,0.049210,0.034067,0.035290,0.049329,0.409516,0.152017,0.145445,0.854506,0.622727,159.658707,5.516435,0.0,10462,1,5,0.876454,0,0,1,1


In [7]:
model = lgb.LGBMRegressor({'device': 'gpu'})

# train_df = train_df.drop(['long_DPR','lat_DPR','pixel','subset'], axis=1)

target = 'precipitation'
col_list = train_df.columns.to_list()
col_list.remove(target)

kfold = KFold(n_splits = 5, random_state = 92)

params = {
        "metric" : 'MAE'}
#         "device" : 'gpu',
#         "gpu_platform_id" : 0,
#         "gpu_device_id" : 0
        

In [ ]:
for fold_, (train_index, val_index) in enumerate(kfold.split(train_df)):
    train_X, val_X = train_df.loc[train_index, col_list],\
                    train_df.loc[val_index, col_list]
    train_y, val_y = train_df.loc[train_index, target], train_df.loc[val_index, target]
    
    train_set = lgb.Dataset(train_X, label = train_y, categorical_feature = ['type','orbit'])
    val_set = lgb.Dataset(val_X, label = val_y, categorical_feature = ['type','orbit'])
    
    model = lgb.train(params, train_set, valid_sets=[train_set, val_set], verbose_eval = 100, early_stopping_rounds = 200)